In [ ]:
import os
import torch
import numpy as np
import pandas as pd

from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score, accuracy_score
from utils.configs import config
from utils.data_loader import get_test_loader, get_data_loaders
from utils.model_list import xception
from utils.graph import draw_confusion_matrix
from torchvision.models.feature_extraction import create_feature_extractor, get_graph_node_names

In [ ]:
device = torch.device('cuda:0')

In [ ]:
model_params = [config.fold]
MODEL_CKPT = os.path.abspath('model_weights/fold_{}/'.format(*model_params))

In [ ]:
net = xception(config=config)

In [ ]:
net.load_state_dict(torch.load(f'model_weights/fold_{config.fold}/model.pth'))

In [ ]:
net.to(device)
net.eval()

In [ ]:
test_dl = get_test_loader(config)

In [ ]:
train_ds, valid_ds = get_data_loaders(config)

In [ ]:
def print_unique(arr):
    unique, counts = np.unique(arr, return_counts=True)
    print(dict(zip(unique, counts)))

In [ ]:
y_true = []
y_pred = []

with torch.no_grad():
    for x, y in valid_ds:
        x = x.to(device)
        out = net(x).detach().cpu().numpy()
        labels = np.asarray(y)

        for logits, label in zip(out, labels):
            y_true.append(label)
            y_pred.append(np.argmax(logits))

In [ ]:
print_unique(y_pred)
print_unique(y_true)

In [ ]:
f1_score(y_true, y_pred, average='macro')

In [ ]:
accuracy_score(y_true, y_pred)

In [ ]:
cm = confusion_matrix(y_true, y_pred, normalize='true')
cm = np.round(cm, 2)

In [ ]:
draw_confusion_matrix(cm, range(16))